<a href="https://colab.research.google.com/github/lumenintellects/seq2seq-chatbot/blob/main/CSK507_EOM_CTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CSK507 End of Module Generative Chatbot


##1. Import and initialise


###1.1 Import
First, import everything needed during pre-processing and training


In [1]:
import datetime
overallStart = datetime.datetime.now()
start = datetime.datetime.now()
print(start)
import locale
print('Prefferred encoding: ',locale.getpreferredencoding())
locale.getpreferredencoding = lambda: "UTF-8"
import os #needed to manipulate the downloaded files within the collab environment and to pull data
import json
from google.colab import userdata
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from matplotlib import pyplot as plt
%matplotlib inline
import spacy
from spacy import displacy
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import zipfile
import spacy.cli
from spacy.lang.en import English # updated
gpu = spacy.prefer_gpu()
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from ipywidgets import IntProgress
from IPython.display import display


end = datetime.datetime.now()


print(end)

print('Time taken to import all packages: ',end-start)

2024-12-05 13:02:26.554254
Prefferred encoding:  UTF-8
2024-12-05 13:02:35.792222
Time taken to import all packages:  0:00:09.237968


##1.2 Download and unzip dataset
We need to pull the dataset down and unzip from https://www.microsoft.com/en-us/download/confirmation.aspx?id=52419

In [2]:
#https://www.microsoft.com/en-us/download/confirmation.aspx?id=52419
start = datetime.datetime.now()
print(start)

if os.path.exists('content/WikiQACorpus.zip') == False:
  print('zip doesn''t exist, downloading')
  !wget -P content https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip
else:
  print('WikiQACorpus.tsv already exists, not need to download')

if os.path.exists('content/WikiQACorpus/WikiQA.tsv') == False:
  print('zip not unzipped, unzipping')
  with zipfile.ZipFile('content/WikiQACorpus.zip', 'r') as zip_ref:
    zip_ref.extractall('content')
else:
  print('WikiQA.tsv already exists, not need to unzip')

end = datetime.datetime.now()

print(end)

print('Time taken to download and unzip: ',end-start)

2024-12-05 13:02:35.802429
zip doesnt exist, downloading
--2024-12-05 13:02:35--  https://download.microsoft.com/download/E/5/F/E5FCFCEE-7005-4814-853D-DAA7C66507E0/WikiQACorpus.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.29.22.44, 2600:1406:bc00:a81::317f, 2600:1406:bc00:a8e::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.29.22.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7094233 (6.8M) [application/octet-stream]
Saving to: ‘content/WikiQACorpus.zip’

WikiQACorpus.zip    100%[===================>]   6.76M  --.-KB/s    in 0.1s    

2024-12-05 13:02:36 (69.8 MB/s) - ‘content/WikiQACorpus.zip’ saved [7094233/7094233]

zip not unzipped, unzipping
2024-12-05 13:02:36.243021
Time taken to download and unzip:  0:00:00.440592


##1.3 Pandas DF
Put data into pandas df

In [3]:
start = datetime.datetime.now()
print(start)

df = pd.read_csv('content/WikiQACorpus/WikiQA.tsv',on_bad_lines='skip',engine='python',sep='\t')
end = datetime.datetime.now()

print(end)

print('Time taken to load to df: ',end-start)

2024-12-05 13:02:36.250869
2024-12-05 13:02:36.436779
Time taken to load to df:  0:00:00.185910


#2. Text normalisation, cleaning and extraction

##2.1 Normalise using NFKD
We first have to nomalise the data using NFKD to make standard alpha so that _é_ becomes _e_ etc. Additionally, check the coding by converting to ascii and back to UTF-8.

In [4]:
start = datetime.datetime.now()
print(start)

df['Question'] = df.Question.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # this will make the text normal alpha, for exaple é will be e, etc
df['Sentence'] = df.Sentence.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8') # this will make the text normal alpha, for exaple é will be e, etc

end = datetime.datetime.now()

print(end)

print('Time taken to nomalise: ',end-start)


2024-12-05 13:02:36.444266
2024-12-05 13:02:36.521948
Time taken to nomalise:  0:00:00.077682


##2.2 Clean data
Next we clean the data by removing triple and double spaces befre making lower case

In [5]:
start = datetime.datetime.now()
print(start)

df['Question'] = df['Question'].str.replace('\n',' ') # remove carriage returns
df['Question'] = df['Question'].str.replace('   ',' ') #remove triple spaces
df['Question'] = df['Question'].str.replace('  ',' ') # also remove double sapces (which might remain after triples are removed)
df['Question'] = df['Question'].str.lower()
df['Sentence'] = df['Sentence'].str.replace('\n',' ') # remove carriage returns
df['Sentence'] = df['Sentence'].str.replace('   ',' ') #remove triple spaces
df['Sentence'] = df['Sentence'].str.replace('  ',' ') # also rmeove double sapces (which might remain after triples are removed)
df['Sentence'] = df['Sentence'].str.lower()

end = datetime.datetime.now()

print(end)

print('Time taken to clean: ',end-start)

2024-12-05 13:02:36.530341
2024-12-05 13:02:36.602939
Time taken to clean:  0:00:00.072598


##2.3 Data Extraction
Next we extract the relevant normalised and cleansed columns of dataframe into ques and ans dataframes

In [6]:
start = datetime.datetime.now()
print(start)


ques = df['Question']
ans = df['Sentence']


end = datetime.datetime.now()
print(end)

print('Time taken to extract: ',end-start)

2024-12-05 13:02:36.609858
2024-12-05 13:02:36.610310
Time taken to extract:  0:00:00.000452


#2.4 Export to csv for team wide review

In [7]:
outputdf = pd.DataFrame({'input': ques, 'output': ans})
outputdf.to_csv('outputdf.csv')

#3. Tokenization and indexing

Next we need to tokenize the data using spaCy before making word-to-index and index-to-word mappings. Finally we will need to convert toindices for the model

##3.1 Import spaCy Pipeline

Large Class used because of copus complexity, en_core_web_lg contains 343k unique vectors, compared with en_core_web_sm which only has 20k

In [8]:
start = datetime.datetime.now()
print(start)

#because of the complex corpus, use the larger en_core_web_lg

try:
  nlp = spacy.load("en_core_web_lg")
except OSError:
  print('Downloading en_core_web_lg')
  spacy.cli.download("en_core_web_lg")
  nlp = spacy.load("en_core_web_lg")

end = datetime.datetime.now()

print(end)

print('Time taken to download and load spaCy class: ',end-start)

2024-12-05 13:02:36.756217
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


2024-12-05 13:02:57.261523
Time taken to download and load spaCy class:  0:00:20.505306


##3.2 Counters

We need a way of getting and storing the frequency, so we need to initalise counters (from collections package)

In [9]:
start = datetime.datetime.now()
print(start)




ques_words = Counter()
ans_words = Counter()


end = datetime.datetime.now()
print(end)

print('Time taken to initialise counters: ',end-start)

2024-12-05 13:02:57.270298
2024-12-05 13:02:57.270493
Time taken to initialise counters:  0:00:00.000195


##3.3 Tokenize

totokenize, we use spaCy pieline already loaded and go through the entire df row by row before stitching _EOS onto the end for the seq-to-seq model later

In [ ]:
start = datetime.datetime.now()
print(start)


gpu = spacy.prefer_gpu() #GPU was found to be slightly better peforming in colab for SpaCy. The GPU selection needs to be immediately before the load command so need to reload
nlp = spacy.load("en_core_web_lg") # large was used

ques_inputs = []
ans_inputs = []

f = IntProgress(min=0, max=len(df)) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
display(f) # display the bar

for i in range(len(df)):
    ques_tokens = nlp(ques[i])
    ans_tokens = nlp(ans[i])




    if (len(ques_tokens)!=0 and len(ans_tokens)!=0):


      for token in ques_tokens:
          ques_words.update([token.text]) #this is the counter for the question frequency, update it


      ques_inputs.append([token.text for token in ques_tokens] + ['_EOS'])

      for token in ans_tokens:
          ans_words.update([token.text]) #this is the counter for the answer frequency, update it

      ans_inputs.append([token.text for token in ans_tokens] + ['_EOS'])
    f.value += 1
end = datetime.datetime.now()

print(end)

print('Time taken to tokenize: ',end-start)



2024-12-05 13:02:57.279234


IntProgress(value=0, max=29019)

##3.4 Vocabulary creation

Next we need to create word-to-index and index-to-word for both questions and answers

In [ ]:

start = datetime.datetime.now()
print(start)


ques_words = ['_SOS','_EOS','_UNK'] + sorted(ques_words,key=ques_words.get,reverse=True) #add 'special words' like EOS to the entire word list
ans_words = ['_SOS','_EOS','_UNK'] + sorted(ans_words,key=ans_words.get,reverse=True)


ques_w2i = {a:i for i,a in enumerate(ques_words)} #record the index against each word with the word first in a dictionary
ans_w2i = {a:i for i,a in enumerate(ans_words)}


ques_i2w = {i:o for i,o in enumerate(ques_words)} #do the same thing but in reverse so the index is first
ans_i2w = {i:o for i,o in enumerate(ans_words)}

g = IntProgress(min=0, max=len(ques_inputs)) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
display(g) # display the bar

for i in range(len(ques_inputs)):
    ques_sentence = ques_inputs[i]
    ans_sentence = ans_inputs[i]
    ques_inputs[i] = [ques_w2i[word] for word in ques_sentence] #this puts columns accross the df to store frequency of each word
    ans_inputs[i] = [ans_w2i[word] for word in ans_sentence]
    g.value += 1

end = datetime.datetime.now()

print(end)

print('Time taken to create vocabulary: ',end-start)

In [ ]:
ques_inputs

#lifted straight from CSK507 coursework:

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, vocab_len, input_dim, hidden_dim, n_layers=1, drop_prob=0):
    super(EncoderLSTM, self).__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers

    self.embedding = nn.Embedding(vocab_len, input_dim)
    self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers,
                        dropout=drop_prob, batch_first=True)

  def forward(self, inputs, encoder_state_vector, encoder_cell_vector):
    embedded = self.embedding(inputs)
    # Pass the embedded word vectors into LSTM and return all outputs
    output, hidden = self.lstm(embedded, (encoder_state_vector, encoder_cell_vector))
    return output, hidden

  def init_hidden(self, batch_size=1):
    return (torch.zeros(self.n_layers, batch_size,
                        self.hidden_dim),
            torch.zeros(self.n_layers, batch_size,
                        self.hidden_dim))

class DecoderLSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_vocab_len, n_layers=1, drop_prob=0.1):
    super(DecoderLSTM, self).__init__()
    self.hidden_dim = hidden_dim
    self.output_vocab_len = output_vocab_len
    self.n_layers = n_layers
    self.drop_prob = drop_prob
    self.input_dim = input_dim

    self.embedding = nn.Embedding(self.output_vocab_len, self.input_dim)
    self.dropout = nn.Dropout(self.drop_prob)
    self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, batch_first=True)
    self.classifier = nn.Linear(self.hidden_dim, self.output_vocab_len)

  def forward(self, inputs, decoder_state_vector, decoder_context_vector):
    # Embed input words
    embedded = self.embedding(inputs).view(1, -1)
    embedded = self.dropout(embedded)
    embedded = embedded.unsqueeze(0)

    output, hidden = self.lstm(embedded, (decoder_state_vector,
                                          decoder_context_vector))

    # Pass LSTM outputs through a Linear layer acting as a classifier
    output = F.log_softmax(self.classifier(output[0]), dim=1)

    return output, hidden



In [ ]:

start = datetime.datetime.now()
print(start)


input_dim = 100
hidden_dim = 256
spacy.prefer_gpu()
encoder = EncoderLSTM(len(ques_words), input_dim, hidden_dim)
decoder = DecoderLSTM(input_dim, hidden_dim, len(ans_words))

lr = 0.001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)

EPOCHS = 10
teacher_forcing_prob = 0.5
encoder.train()
decoder.train()
tk0 = range(1,EPOCHS+1)
for epoch in tk0:
    avg_loss = 0.
    tk1 = enumerate(ques_inputs)
    h = IntProgress(min=0, max=len(tk1)) # instantiate the bar  [[McAteer, S (2017) Stackoverflow: How do I implement a progress bar, avaialable at https://stackoverflow.com/a/41457700 (accessed 25/11/2024)]]
    display(h) # display the bar
    for i, sentence in tk1:

        loss = 0.

        #initialise encoder state vector and cell state vector
        h = encoder.init_hidden()
        encoder_state_vector = h[0]
        encoder_cell_vector = h[0]

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        inp = torch.tensor(sentence).unsqueeze(0)

        #print('inp: ', epoch, inp)
        if (i % 100) == 0:
          print('inp: ', i, epoch)
        encoder_outputs, h = encoder(inp, encoder_state_vector, encoder_cell_vector)

        #First decoder input will be the SOS token
        decoder_input = torch.tensor([ques_w2i['_SOS']])
        #First decoder hidden state will be last encoder hidden state
        decoder_hidden = h

        output = []
        teacher_forcing = True if random.random() < teacher_forcing_prob else False

        for ii in range(len(ans_inputs[i])):
          decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden[0], decoder_hidden[1])

          # Get the index value of the word with the highest score from the decoder output
          top_value, top_index = decoder_output.topk(1)
          if teacher_forcing:
            decoder_input = torch.tensor([ans_inputs[i][ii]])
          else:
            decoder_input = torch.tensor([top_index.item()])

          output.append(top_index.item())
          # Calculate the loss of the prediction against the actual word
          loss += F.nll_loss(decoder_output.view(1,-1), torch.tensor([ans_inputs[i][ii]]))

        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        avg_loss += loss.item()/len(ques_inputs)
        h.value += 1
    print(avg_loss)

# Save model after every epoch (Optional)
torch.save({"encoder":encoder.state_dict(),"decoder":decoder.state_dict(),"q_optimizer":encoder_optimizer.state_dict(),"a_optimizer":decoder_optimizer},"model_enc_dec.pt")


end = datetime.datetime.now()
print(end)

print('Time taken to train model: ',end-start)

In [ ]:

start = datetime.datetime.now()
print(start)

checkpoint = torch.load("model_enc_dec.pt")

encoder.load_state_dict(checkpoint['encoder'])
decoder.load_state_dict(checkpoint['decoder'])
encoder_optimizer.load_state_dict(checkpoint['q_optimizer'])
#decoder_optimizer.load_state_dict(checkpoint['a_optimizer'])

encoder.eval()
decoder.eval()

# get some random numbers to choose random sentences
rand_integers = [random.randint(0, 10000) for i in range(1, 20)]

for i in rand_integers:
  h = encoder.init_hidden()
  inp = torch.tensor(ques_inputs[i]).unsqueeze(0)
  encoder_outputs, h = encoder(inp, h[0], h[1])
  print('inp:',inp)
  print('en_inputs[',i,']: ', ques_inputs[i])
  decoder_input = torch.tensor([ques_w2i['_SOS']])
  decoder_hidden = h
  output = []
  attentions = []
  while True:
    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden[0], decoder_hidden[1])
    _, top_index = decoder_output.topk(1)
    decoder_input = torch.tensor([top_index.item()])
    # If the decoder output is the End Of Sentence token, stop decoding process
    if top_index.item() == ans_w2i["_EOS"]:
      break
    output.append(top_index.item())

  print("Question: "+ " ".join([ques_i2w[x] for x in ques_inputs[i]]))
  print("Answer Predicted: " + " ".join([ans_i2w[x] for x in output]))
  print("Actual: " + " ".join([ans_i2w[x] for x in ans_inputs[i]]))
  print()



end = datetime.datetime.now()
print(end)

print('Time taken to test model: ',end-start)

